<a href="https://colab.research.google.com/github/chandrusuresh/Hands-On-ML/blob/master/Ch2/District_Housing_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tarfile
import urllib

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Data Download

In [2]:
# DOWNLOAD_ROOT = "https://raw.giuthubusercontent.com/ageron/handson-ml2/master/"
# HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# # os.path.exists(/content/drive)